In [6]:
# Generated synthetic univariate or multivariate time series data
import pandas as pd
import numpy as np

def generate_time_series_data(start_time: str = "2021-01-01 00:00:00", 
                              freq: str = "H", 
                              periods: int = 40, 
                              n_variables: int = 10, 
                              univariate: bool = False) -> pd.DataFrame:
    """
    Generate a univariate or multivariate time series DataFrame.
    
    Args:
        start_time (str): Start datetime.
        freq (str): Pandas frequency string (e.g., 'H' for hourly).
        periods (int): Number of time steps.
        n_variables (int): Number of variables (ignored if univariate=True).
        univariate (bool): Whether to generate univariate (single-column) data.

    Returns:
        pd.DataFrame: Time series data with datetime index.
    """
    index = pd.date_range(start=start_time, periods=periods, freq=freq)

    if univariate:
        data = np.random.randn(periods)
        df = pd.DataFrame(data, index=index, columns=["A"])
    else:
        columns = list("ABCDEFGHIJ")[:n_variables]
        data = np.random.randn(periods, n_variables)
        df = pd.DataFrame(data, index=index, columns=columns)
    
    return df

df_multivariate = generate_time_series_data()
print(df_multivariate.columns.values)
df_univariate = generate_time_series_data(univariate=True)
print(df_univariate.columns.values)

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J']
['A']


In [7]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.model.moirai_moe import MoiraiMoEForecast, MoiraiMoEModule

# Model configuration
MODEL = "moirai"  # model name: choose from {'moirai', 'moirai-moe'}
SIZE = "small"  # model size: choose from {'small', 'base', 'large'}
PDT = 4  # prediction length: any positive integer
CTX = 10  # context length: any positive integer
PSZ = 8  # patch size: choose from {"auto", 8, 16, 32, 64, 128}
BSZ = 8  # batch size: any positive integer
TEST = 8  # test set length: any positive integer

gluon_multi = PandasDataset(dict(df_multivariate))
gluon_uni = PandasDataset(dict(df_univariate))
print(gluon_multi)
print(gluon_uni)

/opt/miniconda3/envs/moirai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PandasDataset<size=10, freq=H, num_feat_dynamic_real=0, num_past_feat_dynamic_real=0, num_feat_static_real=0, num_feat_static_cat=0, static_cardinalities=[]>
PandasDataset<size=1, freq=H, num_feat_dynamic_real=0, num_past_feat_dynamic_real=0, num_feat_static_real=0, num_feat_static_cat=0, static_cardinalities=[]>


In [8]:
# Obtain train and test data
train_multi, test_template_multi = split(
    gluon_multi, offset=-TEST
)  # assign last TEST time steps as test set


train_uni, test_template_uni = split(
    gluon_uni, offset=-TEST
)  # assign last TEST time steps as test set


# Construct rolling window evaluation
test_data_multi = test_template_multi.generate_instances(
    prediction_length=PDT,  # number of time steps for each prediction
    windows=TEST // PDT,  # number of windows in rolling window evaluation
    distance=PDT,  # number of time steps between each window - distance=PDT for non-overlapping windows
)

# Construct rolling window evaluation
test_data_uni = test_template_uni.generate_instances(
    prediction_length=PDT,  # number of time steps for each prediction
    windows=TEST // PDT,  # number of windows in rolling window evaluation
    distance=PDT,  # number of time steps between each window - distance=PDT for non-overlapping windows
)



In [9]:
def moirai_maker(MODEL, gluon_data):
    if MODEL == "moirai":
        model = MoiraiForecast(
            module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.1-R-{SIZE}"),
            prediction_length=PDT,
            context_length=CTX,
            patch_size=PSZ,
            num_samples=5,
            target_dim=1,
            feat_dynamic_real_dim=gluon_data.num_feat_dynamic_real,
            past_feat_dynamic_real_dim=gluon_data.num_past_feat_dynamic_real,
        )
    elif MODEL == "moirai-moe":
        model = MoiraiMoEForecast(
            module=MoiraiMoEModule.from_pretrained(f"Salesforce/moirai-moe-1.0-R-{SIZE}"),
            prediction_length=PDT,
            context_length=CTX,
            patch_size=PSZ,
            num_samples=5,
            target_dim=1,
            feat_dynamic_real_dim=gluon_data.num_feat_dynamic_real,
            past_feat_dynamic_real_dim=gluon_data.num_past_feat_dynamic_real,
        )
    return model

In [10]:
# Moirai-MoE multivariate
model = moirai_maker("moirai-moe", gluon_multi)
predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data_multi.input)

input_it = iter(test_data_multi.input)
label_it = iter(test_data_multi.label)
forecast_it = iter(forecasts)

#inp = next(input_it)
#label = next(label_it)
#forecast = next(forecast_it)
results_dict = dict()
letter_sequence = "ABCDEFGHIJ"
for letter in letter_sequence:
    results_dict[letter] = []
    
for fore in forecast_it:
    #print(fore)
    for letter in letter_sequence:
        if fore.item_id == letter:
            #print(fore.samples)
            results_dict[letter].extend(np.median(fore.samples,axis=0))


print(results_dict)

{'A': [-0.06505018, -0.0647412, 0.028948188, -0.06505018, -0.28263474, -0.28158107, -0.2837789, -0.2824344], 'B': [0.14207464, 0.14207464, 0.14373998, 0.14207464, -0.18114492, 0.3115472, 1.0491917, 0.71172345], 'C': [0.39336014, 0.38719013, 0.38810328, 0.386574, 0.29872653, 0.3426705, 0.85204697, 0.1361086], 'D': [0.4198732, 0.09615295, 0.0970043, 0.0970043, 0.33982685, 0.7038663, 0.7466946, 0.04396508], 'E': [0.29222387, 0.028680222, 0.028064772, 0.8402004, -0.13939965, 0.20156321, -0.1373384, -0.13881038], 'F': [-0.3484138, -0.34806964, -0.3485902, -0.3485902, -0.28533745, 0.849535, 0.4582345, -0.47556353], 'G': [0.3313228, 0.8439627, 0.78163177, 0.8877248, 0.30575645, 0.29905587, -0.21102305, 0.44750923], 'H': [0.43077624, 0.77461547, -0.18945564, -0.18962549, 0.037747025, 0.012781054, -0.47052932, -0.47052932], 'I': [0.03618395, 0.54177225, 0.50242376, 0.03542605, 0.12670133, 0.9145477, 0.1263691, 0.45548034], 'J': [0.6180278, 0.07978705, 0.27223608, 0.620325, -0.19415092, -0.19292

In [11]:
# Moirai multivariate
model = moirai_maker("moirai", gluon_multi)
predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data_multi.input)

input_it = iter(test_data_multi.input)
label_it = iter(test_data_multi.label)
forecast_it = iter(forecasts)

#inp = next(input_it)
#label = next(label_it)
#forecast = next(forecast_it)
results_dict = dict()
letter_sequence = "ABCDEFGHIJ"
for letter in letter_sequence:
    results_dict[letter] = []
    
for fore in forecast_it:
    #print(fore)
    for letter in letter_sequence:
        if fore.item_id == letter:
            #print(fore.samples)
            results_dict[letter].extend(np.median(fore.samples,axis=0))


print(results_dict)

{'A': [0.044499822, -0.037347313, -0.33144373, -0.76090187, -0.42878094, -0.024220198, -0.58256596, 0.0016886294], 'B': [0.034351543, 0.09169806, 1.2725878, 0.7167516, 0.19907618, -0.7321631, -0.5418392, 0.02140984], 'C': [0.5145378, 0.2939791, 0.36652154, 0.2860617, 1.1174388, 1.9552984, -1.3402965, 1.5847049], 'D': [-0.16374317, -0.19149417, -0.17374855, -0.16588548, 0.1996063, 0.12298791, -0.38323662, 1.807612], 'E': [0.69974107, -0.1468933, -0.14568925, 0.05614644, -0.29418612, -0.32523054, 0.13252795, -0.38278723], 'F': [-0.33108, 0.9309942, -1.1684952, -0.10930133, 0.23793828, -0.50864834, 0.7781533, -0.027074635], 'G': [0.2554523, 0.94483644, 0.7492728, 0.577148, 0.040455982, -0.8086947, -1.227626, 1.3105298], 'H': [0.17397054, -0.68440795, 3.1789086, -0.76023483, -1.5583054, -1.6864871, -1.5639361, -0.42076182], 'I': [0.14843222, -0.3391226, -0.047155328, -0.3826533, 0.001131013, -0.43334877, 1.8297436, -0.80715203], 'J': [-0.0075924546, 0.33318695, 0.1569424, 1.0832374, -0.860

In [12]:
# Moirai-MoE univariate
model = moirai_maker("moirai-moe", gluon_uni)
predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data_uni.input)

input_it = iter(test_data_uni.input)
label_it = iter(test_data_uni.label)
forecast_it = iter(forecasts)

#inp = next(input_it)
#label = next(label_it)
#forecast = next(forecast_it)
results_dict = dict()
letter_sequence = "A"
for letter in letter_sequence:
    results_dict[letter] = []
    
for fore in forecast_it:
    #print(fore)
    for letter in letter_sequence:
        if fore.item_id == letter:
            #print(fore.samples)
            results_dict[letter].extend(np.median(fore.samples,axis=0))


print(results_dict)

{'A': [0.59439766, 0.13294606, 0.6938918, 0.13253312, 0.17124592, 0.16950361, 0.16950361, 0.16966084]}


In [13]:
# Moirai univariate
model = moirai_maker("moirai", gluon_uni)
predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data_uni.input)

input_it = iter(test_data_uni.input)
label_it = iter(test_data_uni.label)
forecast_it = iter(forecasts)

#inp = next(input_it)
#label = next(label_it)
#forecast = next(forecast_it)
results_dict = dict()
letter_sequence = "A"
for letter in letter_sequence:
    results_dict[letter] = []
    
for fore in forecast_it:
    #print(fore)
    for letter in letter_sequence:
        if fore.item_id == letter:
            #print(fore.samples)
            results_dict[letter].extend(np.median(fore.samples,axis=0))


print(results_dict)

{'A': [0.26503533, -0.042675167, 0.33016336, 0.28003436, 0.7121524, -1.3061789, 0.17313294, 2.8143198]}
